# 中厚板指数

### 爬取参数
- 江西萍钢 江阴市 普中板 Q235B 14-20mm
- 江西萍钢 杭州市 普中板 Q235B 14-20mm

- 江西萍钢 二级目录入口  https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html
- 名称：25日江阴市场中厚板价格行情  
    - 日期 + 日江阴市场中厚板价格行情  
- 普中板	14-20	Q235B	萍钢	3760
- "ctr15"


- 杭州萍钢 二级目录入口  https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html
- 名称：25日杭州市场中厚板价格行情
    - 日期 + 日杭州市场中厚板价格行情
- 普中板 14-20 Q235B 萍钢 3760 -10 现货
- "ctr10"

- 上海鞍钢/本钢 热轧板卷 Q235B 5.5-11.75*1500*C
    - 二级入口：https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html
    - 名称：1日（09：35）上海市场热轧板卷价格行情 // 或9:30
    - ctr50
    - "热轧板卷 5.5-11.75*1500*C Q235B 鞍钢/本钢"
- 张家港沙钢 热轧板卷 Q235B 5.75*1500*C
    - 二级入口：https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html
    - 名称：6日张家港市场热轧板卷价格行情
    - ctr13
    - 热轧板卷 
    - "热轧板卷 5.75*1500*C Q235B 沙钢"

    

In [1]:
# -*- coding: utf-8 -*-
import os
import requests

import time
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# 常量
CHROME_DRIVER = 'D:\Python\驱动\chromedriver_win32\chromedriver.exe'

## 设置条件参数
date_today = str(datetime.datetime.now().day)  # 取系统时间
date_today = str(2)
para_dic = {0:{
               "二级入口网址":"https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html",
               "二级入口网址查找对应表名":date_today + "日江阴市场中厚板价格行情",
               "每日新增数据的网址":None, 
               "判断条件":"普中板 14-20 Q235B 萍钢",
               "CTR值":"ctr15" },
            1:{
              "二级入口网址":"https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html",
              "二级入口网址查找对应表名":date_today + "日杭州市场中厚板价格行情",
              "每日新增数据的网址":None, 
              "判断条件":"普中板 14-20 Q235B 萍钢",
              "CTR值":"ctr10"},
            2:{
              "二级入口网址":"https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html",
              "二级入口网址查找对应表名":date_today + "日（09：35）上海市场热轧板卷价格行情",
              "每日新增数据的网址":None, 
              "判断条件":"热轧板卷 5.5-11.75*1500*C Q235B 鞍钢/本钢",
              "CTR值":"ctr50"}, 
            3:{
              "二级入口网址":"https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html",
              "二级入口网址查找对应表名":date_today + "日（09：30）上海市场热轧板卷价格行情",
              "每日新增数据的网址":None, 
              "判断条件":"热轧板卷 5.5-11.75*1500*C Q235B 鞍钢/本钢",
              "CTR值":"ctr50"}, 
            4:{
              "二级入口网址":"https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html",
              "二级入口网址查找对应表名":date_today + "日张家港市场热轧板卷价格行情",
              "每日新增数据的网址":None, 
              "判断条件":"热轧板卷 5.75*1500*C Q235B 沙钢",
              "CTR值":"ctr13"},
           }

In [3]:
## 抓取对应的网页信息
def get_specfic_table_link(table_content,match_title):
    match_a = table_content.findAll("a",title=match_title)  # 在获取到的页面的bs4对象上，继续找到对应a标签对象
    
    next_a_href_suffix = table_content.findAll("a",title=match_title)[0].get("href")
    if next_a_href_suffix.startswith("http"):
        next_a_href = next_a_href_suffix
    else:
        next_a_href = "https:" + next_a_href_suffix
    return next_a_href

def find_content_href(index,para_dic=para_dic):
    url = para_dic.get(index).get("二级入口网址")
    match_title = para_dic.get(index).get("二级入口网址查找对应表名")
    response = requests.get(url)
    html_doc = response.text 
    soup = BeautifulSoup(html_doc,"html.parser")
    table_content = soup.find("ul",class_="nlist")
    
    # 判断是否在第一页就能找到对应的 表名 a标签
    titles = table_content.get_text().replace("\t","").replace("\n","")
    
    if titles.find(match_title) == -1:
        # 找第二页
        find_index = soup.find("div",class_="page")
        page2 = find_index.find("a")
        if page2.contents[0] == "2":
            page2_change = find_index.find("a")["href"].split("-")[-1]
            page2_url = url.replace("1.html",page2_change)
            response2 = requests.get(page2_url)
            soup2 = BeautifulSoup(response2.text,"html.parser")
            table_content2 = soup2.find("ul",class_="nlist")
            titles2 = table_content2.get_text().replace("\t","").replace("\n","")
            if titles2.find(match_title) == -1:
                print("今日数据尚未更新")
                return para_dic
            else: 
                para_dic[index]["是否找到对应表名"] = True
                para_dic[index]["每日新增数据的网址"] = get_specfic_table_link(table_content2,match_title)
    else:
        # 如果找到了指定的表名
        # 则解析获取对应的a标签的href网址
        # 返回对应的表名的 href 字符串
        para_dic[index]["是否找到对应表名"] = True
        para_dic[index]["每日新增数据的网址"] = get_specfic_table_link(table_content,match_title) 
        return para_dic

In [4]:
for i in range(len(para_dic)):
    find_content_href(i)
    print(para_dic.get(i).get("是否找到对应表名"),para_dic.get(i).get("每日新增数据的网址"))

True https://zhongban.mysteel.com/m/20/0702/09/590834486A5B7D41.html
True https://zhongban.mysteel.com/m/20/0702/09/1A774041E25E79A9.html
今日数据尚未更新
None None
True https://rezha.mysteel.com/m/20/0702/09/S07A221845CF34644.html
True https://rezha.mysteel.com/m/20/0702/09/3544C2C7F279407D.html


In [5]:
para_dic

{0: {'二级入口网址': 'https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日江阴市场中厚板价格行情',
  '每日新增数据的网址': 'https://zhongban.mysteel.com/m/20/0702/09/590834486A5B7D41.html',
  '判断条件': '普中板 14-20 Q235B 萍钢',
  'CTR值': 'ctr15',
  '是否找到对应表名': True},
 1: {'二级入口网址': 'https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日杭州市场中厚板价格行情',
  '每日新增数据的网址': 'https://zhongban.mysteel.com/m/20/0702/09/1A774041E25E79A9.html',
  '判断条件': '普中板 14-20 Q235B 萍钢',
  'CTR值': 'ctr10',
  '是否找到对应表名': True},
 2: {'二级入口网址': 'https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日（09：35）上海市场热轧板卷价格行情',
  '每日新增数据的网址': None,
  '判断条件': '热轧板卷 5.5-11.75*1500*C Q235B 鞍钢/本钢',
  'CTR值': 'ctr50'},
 3: {'二级入口网址': 'https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日（09：30）上海市场热轧板卷价格行情',
  '每日新增数据的网址': 'https://rezha.mysteel.com/m/20/0702/09/S07A221845CF34644.html',
  '判断条件': '热轧板卷 

In [6]:
#指定chrome的驱动
#执行到这里的时候Selenium会到指定的路径将chrome driver程序运行起来
driver = webdriver.Chrome(CHROME_DRIVER)
if para_dic.get(0).get("每日新增数据的网址") is not None:
    driver.get(para_dic.get(1).get("每日新增数据的网址"))
else:
    driver.get("https://zhongban.mysteel.com/m/20/0525/10/F21EFCF6E0FA1977.html")
    
time.sleep(10)
for k,v in para_dic.items():
    #get 方法 打开指定网址
    if para_dic.get(k).get("每日新增数据的网址") is not None:
            driver.get(para_dic.get(k).get("每日新增数据的网址"))
            id_ctr_value = para_dic.get(k).get("CTR值")
            #选择网页元素
            element_keyword = driver.find_element_by_id(id_ctr_value)
            para_dic[k]["匹配原始数据值"]=element_keyword.text
        
    time.sleep(1)
    print(para_dic[k])

{'二级入口网址': 'https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html', '二级入口网址查找对应表名': '2日江阴市场中厚板价格行情', '每日新增数据的网址': 'https://zhongban.mysteel.com/m/20/0702/09/590834486A5B7D41.html', '判断条件': '普中板 14-20 Q235B 萍钢', 'CTR值': 'ctr15', '是否找到对应表名': True, '匹配原始数据值': '普中板 14-20 Q235B 萍钢 3820 - 货少 代理'}
{'二级入口网址': 'https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html', '二级入口网址查找对应表名': '2日杭州市场中厚板价格行情', '每日新增数据的网址': 'https://zhongban.mysteel.com/m/20/0702/09/1A774041E25E79A9.html', '判断条件': '普中板 14-20 Q235B 萍钢', 'CTR值': 'ctr10', '是否找到对应表名': True, '匹配原始数据值': '普中板 14-20 Q235B 萍钢 3830 - 现货'}
{'二级入口网址': 'https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html', '二级入口网址查找对应表名': '2日（09：35）上海市场热轧板卷价格行情', '每日新增数据的网址': None, '判断条件': '热轧板卷 5.5-11.75*1500*C Q235B 鞍钢/本钢', 'CTR值': 'ctr50'}
{'二级入口网址': 'https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html', '二级入口网址查找对应表名': '2日（09：30）上海市场热轧板卷价格行情', '每日新增数据的网址': 'https://rezha.mysteel.com/m/20/0702/09/S07

In [7]:
para_dic

{0: {'二级入口网址': 'https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日江阴市场中厚板价格行情',
  '每日新增数据的网址': 'https://zhongban.mysteel.com/m/20/0702/09/590834486A5B7D41.html',
  '判断条件': '普中板 14-20 Q235B 萍钢',
  'CTR值': 'ctr15',
  '是否找到对应表名': True,
  '匹配原始数据值': '普中板 14-20 Q235B 萍钢 3820 - 货少 代理'},
 1: {'二级入口网址': 'https://list1.mysteel.com/market/p-220-----010102-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日杭州市场中厚板价格行情',
  '每日新增数据的网址': 'https://zhongban.mysteel.com/m/20/0702/09/1A774041E25E79A9.html',
  '判断条件': '普中板 14-20 Q235B 萍钢',
  'CTR值': 'ctr10',
  '是否找到对应表名': True,
  '匹配原始数据值': '普中板 14-20 Q235B 萍钢 3830 - 现货'},
 2: {'二级入口网址': 'https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日（09：35）上海市场热轧板卷价格行情',
  '每日新增数据的网址': None,
  '判断条件': '热轧板卷 5.5-11.75*1500*C Q235B 鞍钢/本钢',
  'CTR值': 'ctr50'},
 3: {'二级入口网址': 'https://list1.mysteel.com/market/p-231-----010103-0-0101-------1.html',
  '二级入口网址查找对应表名': '2日（09：30）上海市场热轧板卷价格行情',
  '

In [8]:
# 根据爬取的“匹配原始数据值” 与 “判断条件” 进行比较，确认爬取的记录是正确的
def check_data(index,original,condition):
    # original 是 爬取的数据，“匹配原始数据值”
    # condition 是 “判断条件”
    o_list = original.split()
    c1,c2,c3,c4 = condition.split()
    
    if c1 == o_list[0] and c2 == o_list[1] and c3 == o_list[2] and c4 == o_list[3]:
        print(para_dic.get(index).get("二级入口网址查找对应表名") +"     "+ o_list[4])
    else:
        print("有错误，建议查找")
        

In [9]:
for i in range(len(para_dic)):
    original = para_dic.get(i).get("匹配原始数据值")
    condition = para_dic.get(i).get("判断条件")
    if original:
        check_data(i,original,condition)
    else:
        print("NoneType值，不能进行校验")

2日江阴市场中厚板价格行情     3820
2日杭州市场中厚板价格行情     3830
NoneType值，不能进行校验
2日（09：30）上海市场热轧板卷价格行情     3730
2日张家港市场热轧板卷价格行情     3730
